In [1]:
from dotenv import load_dotenv
from database import Database
import pandas as pd
import os
import mysql.connector
load_dotenv()
db = Database() 
conn = mysql.connector.connect(
    host=os.getenv("DB_HOST"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    database="cricsheet_db"
)
cursor = conn.cursor()

####  (1) Top 10 batsmen by total runs in ODI

In [2]:
query  = """
        SELECT batsman ,sum(runs_batsman) as total_run FROM ODI_Match
        group by batsman order by total_run desc limit 10 ; """

cursor.execute(query)
results = cursor.fetchall()

db.close_connection()
coln = [desc[0] for desc in cursor.description]
df = pd.DataFrame(results, columns=coln)
print(df)

            batsman total_run
0           V Kohli     28118
1     KC Sangakkara     23236
2         RG Sharma     21896
3          MS Dhoni     20548
4    AB de Villiers     18870
5        TM Dilshan     18424
6       LRPL Taylor     16252
7  DPMD Jayawardene     16080
8           HM Amla     15668
9       Tamim Iqbal     15296


#### (2)Leading wicket-takers in T20 matches

In [3]:
query  = """  select bowler, count(*) as total_wickets from t20_match 
    where runs_batsman =0 group by bowler order by total_wickets desc limit 15   """

cursor.execute(query)
results = cursor.fetchall()

db.close_connection()
coln = [desc[0] for desc in cursor.description]
df = pd.DataFrame(results, columns=coln)
print(df)

               bowler  total_wickets
0          TG Southee           2480
1           ML Schutt           2388
2           DB Sharma           2344
3            Nida Dar           2246
4        S Ecclestone           2201
5   Mustafizur Rahman           2076
6           W Mwatile           2014
7            KH Brunt           1968
8           AU Rashid           1942
9              M Kapp           1942
10           S Ismail           1940
11      C Sutthiruang           1925
12    Shakib Al Hasan           1922
13         MJ Santner           1922
14       N Boochatham           1920


#### (3)Team with the highest win percentage in Test cricket.

In [4]:
query  = """   select match_winner, count(*) as total_wins,round((count(*)/(select count(*) from test_match )* 100),2) as win_percentage from test_match
        group by match_winner order by win_percentage desc limit 10  """

cursor.execute(query)
results = cursor.fetchall()

db.close_connection()
coln = [desc[0] for desc in cursor.description]
df = pd.DataFrame(results, columns=coln)
print(df)

   match_winner  total_wins win_percentage
0          None      754930          22.52
1     Australia      487946          14.56
2       England      438217          13.07
3         India      388868          11.60
4  South Africa      331529           9.89
5     Sri Lanka      256448           7.65
6   New Zealand      233198           6.96
7      Pakistan      214330           6.39
8   West Indies      140412           4.19
9    Bangladesh       81468           2.43


#### (4) Top 10 Players with Highest Career Runs Across All Formats

In [5]:
query  = """select batsman, sum(runs_batsman) as total_runs from 
            (select batsman, runs_batsman from test_match union all
            select batsman, runs_batsman from odi_match union all
            select batsman, runs_batsman from t20_match union all
            select batsman, runs_batsman from ipl_match) as combined
        group by batsman order by total_runs desc limit 10; """

cursor.execute(query)
results = cursor.fetchall()

db.close_connection()
coln = [desc[0] for desc in cursor.description]
df = pd.DataFrame(results, columns=coln)
print(df)

          batsman total_runs
0         V Kohli      71449
1       RG Sharma      52342
2       DA Warner      50460
3  AB de Villiers      48778
4   KC Sangakkara      46352
5        MS Dhoni      43970
6   KS Williamson      42138
7         JE Root      41330
8       SPD Smith      39127
9         HM Amla      37658


#### (5)Matches with the narrowest margin of victory.

In [6]:
query  = """ select match_id, match_winner, min(win_by_runs) as min_win_margin from ( select  match_id, match_winner, win_by_runs from test_match where win_by_runs is not null  
            union  ALL
            select  match_id, match_winner, win_by_runs from t20_match where win_by_runs is not null 
            union all 
            select  match_id, match_winner, win_by_runs from odi_match where win_by_runs is not null
            union all 
            select  match_id, match_winner, win_by_runs from ipl_match where win_by_runs is not null
            ) as combined_result
            GROUP BY match_id, match_winner  order by min_win_margin asc limit 10;  """

cursor.execute(query)
results = cursor.fetchall()

db.close_connection()
coln = [desc[0] for desc in cursor.description]
df = pd.DataFrame(results, columns=coln)
print(df)

  match_id  match_winner  min_win_margin
0  1263153  South Africa               1
1  1185313  South Africa               1
2  1138196   New Zealand               1
3  1172165       England               1
4  1256721       England               1
5  1041615   West Indies               1
6  1289043         Kenya               1
7  1282748         Italy               1
8  1322356   New Zealand               1
9  1184902         Kenya               1


#### (6)Total number of matches played by each team

In [7]:
query  = """  select team, count(*) as total_matches from (select DISTINCT team_1 AS team, match_id  from test_match union all select DISTINCT team_2 AS team, match_id  from test_match union all  
                select DISTINCT team_1 AS team, match_id  from t20_match union all select DISTINCT team_2 AS team, match_id  from t20_match union select DISTINCT team_1 AS team, match_id  from odi_match union all
                select DISTINCT team_2 AS team, match_id  from odi_match union all
                select DISTINCT team_1 AS team, match_id  from ipl_match union all  select DISTINCT team_2 AS team, match_id  from ipl_match ) as combined 
                group by team order by total_matches desc limit 10;"""

cursor.execute(query)
results = cursor.fetchall()

db.close_connection()
coln = [desc[0] for desc in cursor.description]
df = pd.DataFrame(results, columns=coln)
print(df)

           team  total_matches
0         India           1224
1       England           1203
2     Australia           1151
3      Pakistan           1042
4     Sri Lanka           1040
5  South Africa           1030
6   New Zealand            993
7   West Indies            974
8    Bangladesh            761
9      Zimbabwe            546


#### (7) Win/loss ratio of each team in ODIs

In [8]:
query  = """ SELECT match_winner AS team, COUNT(*) AS total_wins, (COUNT(*) / (SELECT COUNT(*) FROM odi_match WHERE match_winner IS NOT NULL)) * 100 AS win_percentage
                FROM odi_match WHERE match_winner IS NOT NULL
                GROUP BY match_winner ORDER BY total_wins DESC limit 10;    """

cursor.execute(query)
results = cursor.fetchall()

db.close_connection()
coln = [desc[0] for desc in cursor.description]
df = pd.DataFrame(results, columns=coln)
print(df)

           team  total_wins win_percentage
0     Australia      415046        13.6254
1         India      402893        13.2264
2       England      319313        10.4826
3  South Africa      317915        10.4367
4   New Zealand      261348         8.5797
5      Pakistan      256818         8.4310
6     Sri Lanka      253299         8.3155
7   West Indies      185652         6.0947
8    Bangladesh      147066         4.8280
9      Zimbabwe       76852         2.5229


#### (8)Most successful chasing team in T20s (Highest wins while chasing)

In [9]:
query  = """ select match_winner, COUNT(*) as chases_won from t20_match where toss_decision = 'field' group by match_winner
        order by chases_won desc limit 10;"""

cursor.execute(query)
results = cursor.fetchall()

db.close_connection()
coln = [desc[0] for desc in cursor.description]
df = pd.DataFrame(results, columns=coln)
print(df)

   match_winner  chases_won
0         India       70450
1       England       58866
2     Australia       55510
3   New Zealand       50356
4  South Africa       44755
5      Pakistan       42597
6   West Indies       36508
7     Sri Lanka       31212
8       Ireland       27878
9    Bangladesh       26835


#### (9) Correlation between toss winners and match winners

In [10]:
query  = """ select count(*) as total_matches, sum(case when toss_winner = match_winner then 1 else 0 end)  as toss_win_match_win,
            (sum(case when toss_winner = match_winner then 1 else 0 end) /count(*)) * 100 as correlation_percentage
            from (select toss_winner, match_winner from test_match union all select toss_winner, match_winner from odi_match union all
            select toss_winner, match_winner from t20_match union all select toss_winner, match_winner from ipl_match) AS combined;"""

cursor.execute(query)
results = cursor.fetchall()

db.close_connection()
coln = [desc[0] for desc in cursor.description]
df = pd.DataFrame(results, columns=coln)
print(df)

   total_matches toss_win_match_win correlation_percentage
0        8937344            4176625                46.7323


#### (10)Most common toss decision in ODIs (bat/field)?

In [11]:
query  = """ select toss_decision, count(*) AS count from odi_match group by toss_decision order by count desc;"""

cursor.execute(query)
results = cursor.fetchall()

db.close_connection()
coln = [desc[0] for desc in cursor.description]
df = pd.DataFrame(results, columns=coln)
print(df)

  toss_decision    count
0           bat  1570293
1         field  1562677


#### (11) Player with the most "Player of the Match" awards in ODIs

In [12]:
query  = """ select player_of_match, COUNT(*) AS awards from odi_match where player_of_match !='Unknown'  
                group by player_of_match order by awards desc limit 10;"""

cursor.execute(query)
results = cursor.fetchall()

db.close_connection()
coln = [desc[0] for desc in cursor.description]
df = pd.DataFrame(results, columns=coln)
print(df)

   player_of_match  awards
0          V Kohli   47972
1    KC Sangakkara   30426
2   AB de Villiers   29662
3        RG Sharma   28646
4       TM Dilshan   25868
5  Shakib Al Hasan   23842
6       MJ Guptill   22850
7         MS Dhoni   22762
8     Yuvraj Singh   22238
9  Mohammad Hafeez   21826


#### (12)Team with Most Matches Played Across All Formats

In [13]:
query  = """ select team, COUNT(*) as total_matches from
  ( select team_1 as team from test_match  union all  
    select team_2 as team from test_match  union all  
    select team_1 as team from odi_match  union all  
    select team_2 as team from odi_match  union all  
    select team_1 as team from t20_match  union all  
    select team_2 as team from t20_match  union all  
    select team_1 as team from ipl_match  union all  
    select team_2 as team from ipl_match) as combined 
	group by team order by  total_matches desc limit 10;"""

cursor.execute(query)
results = cursor.fetchall()

db.close_connection()
coln = [desc[0] for desc in cursor.description]
df = pd.DataFrame(results, columns=coln)
print(df)

           team  total_matches
0       England        1841327
1     Australia        1690601
2         India        1685264
3  South Africa        1392007
4     Sri Lanka        1390161
5      Pakistan        1347972
6   New Zealand        1291478
7   West Indies        1249595
8    Bangladesh         931845
9      Zimbabwe         577797


#### (13) Highest individual score in an ODI match

In [14]:
query  = """select batsman, match_id, sum(runs_batsman) as highest_score from odi_match
            group by batsman, match_id order by highest_score desc limit 1 """

cursor.execute(query)
results = cursor.fetchall()

db.close_connection()
coln = [desc[0] for desc in cursor.description]
df = pd.DataFrame(results, columns=coln)
print(df)

     batsman match_id highest_score
0  RG Sharma   792295           528


#### (14)Venues with the highest average runs per match 

In [15]:
query  = """select venue,cast(avg(runs_total) as UNSIGNED) as avg_total_runs 
            FROM (select venue, match_id, sum(runs_total) as runs_total from test_match group by  venue, match_id union all
            select venue, match_id, sum(runs_total) as runs_total from odi_match group by venue, match_id union all
            select venue, match_id, sum(runs_total) as runs_total from t20_match group by venue, match_id union all
            select venue, match_id, sum(runs_total) as runs_total from ipl_match group by venue, match_id) as combined 
            group by venue order by avg_total_runs desc limit 10;"""

cursor.execute(query)
results = cursor.fetchall()

db.close_connection()
coln = [desc[0] for desc in cursor.description]
df = pd.DataFrame(results, columns=coln)
print(df)

                                               venue  avg_total_runs
0                          Antigua Recreation Ground            2920
1      Antigua Recreation Ground, St John's, Antigua            2630
2  Punjab Cricket Association Stadium, Mohali, Ch...            2380
3                               Sardar Patel Stadium            2338
4     Daren Sammy National Cricket Stadium, St Lucia            2088
5                                    Kensington Oval            2020
6                               Sabina Park, Jamaica            1822
7                               Vidarbha C.A. Ground            1821
8  Darren Sammy National Cricket Stadium, Gros Islet            1806
9                                 Green Park, Kanpur            1802


#### (15)Best bowling figures in a single ODI match (most wickets in a match)

In [16]:
query  = """ select bowler, match_id, count(*) as wickets from odi_match where runs_batsman = 0
            group by bowler, match_id order by wickets desc limit 5;"""

cursor.execute(query)
results = cursor.fetchall()

db.close_connection()
coln = [desc[0] for desc in cursor.description]
df = pd.DataFrame(results, columns=coln)
print(df)

             bowler match_id  wickets
0     Mohammad Sami    66363      124
1     Shabbir Ahmed    64883      122
2       BA Williams    66366      120
3         DB Sharma  1073427      120
4  Shahadat Hossain   264070      118


#### (16)Highest partnership (sum of consecutive batsmen runs) in T20s

In [17]:
query  = """select batsman, SUM(runs_batsman) as total_runs from t20_match
group by batsman order by total_runs DESC limit 10;"""

cursor.execute(query)
results = cursor.fetchall()

db.close_connection()
coln = [desc[0] for desc in cursor.description]
df = pd.DataFrame(results, columns=coln)
print(df)

           batsman total_runs
0       Babar Azam       8344
1        RG Sharma       8084
2          V Kohli       7938
3       JC Buttler       7159
4       S Mandhana       7038
5       MJ Guptill       7034
6  Mohammad Rizwan       6772
7        BL Mooney       6674
8        DA Warner       6508
9         SW Bates       6330


#### (17)Most Wickets in a Single IPL Match

In [18]:
query  = """select match_id,bowler, COUNT(*) as wickets from ipl_match where runs_batsman = 0 
        group by match_id, bowler order by wickets desc limit 10;"""

cursor.execute(query)
results = cursor.fetchall()

db.close_connection()
coln = [desc[0] for desc in cursor.description]
df = pd.DataFrame(results, columns=coln)
print(df)

  match_id          bowler  wickets
0  1304105         DR Sams       48
1  1359479  Mohammed Siraj       44
2   392222      SL Malinga       44
3  1422132       JJ Bumrah       42
4   336012        DW Steyn       42
5  1304082     Umran Malik       42
6   392210      SM Harwood       42
7   336023   Sohail Tanvir       42
8  1178398       DL Chahar       42
9   392195     Kamran Khan       42


#### (18)Number of matches played at each venue

In [19]:
query  = """select venue, count(distinct match_id) as total_matches from
            (select venue, match_id from test_match union all  select venue, match_id from odi_match union all 
            select venue, match_id from t20_match union all select venue, match_id from ipl_match) as combined
            group by venue order by total_matches desc limit 10; """

cursor.execute(query)
results = cursor.fetchall()

db.close_connection()
coln = [desc[0] for desc in cursor.description]
df = pd.DataFrame(results, columns=coln)
print(df)

                                               venue  total_matches
0                Dubai International Cricket Stadium            214
1                                 Harare Sports Club            191
2                      Shere Bangla National Stadium            136
3  Al Amerat Cricket Ground Oman Cricket (Ministr...            129
4                               Sheikh Zayed Stadium            116
5                                       Eden Gardens            103
6                            Sharjah Cricket Stadium            100
7                                   Wankhede Stadium             97
8                              Sydney Cricket Ground             94
9                              M Chinnaswamy Stadium             91


#### (19)Most common toss decision in ODIs

In [20]:
query  = """select toss_decision, count(*) as count from odi_match 
group by toss_decision order by count desc;"""

cursor.execute(query)
results = cursor.fetchall()

db.close_connection()
coln = [desc[0] for desc in cursor.description]
df = pd.DataFrame(results, columns=coln)
print(df)

  toss_decision    count
0           bat  1570293
1         field  1562677


#### (20) Best Finisher in T20 & IPL Matches

In [21]:
query  = """select batsman, SUM(runs_batsman) as total_runs_in_death_overs from 
            (select batsman, runs_batsman from t20_match where over_number between 16 and 20 union all
            select batsman, runs_batsman from ipl_match where over_number between 16 and 20 ) as combined
            group by batsman order by total_runs_in_death_overs desc limit 10;"""

cursor.execute(query)
results = cursor.fetchall()

db.close_connection()
coln = [desc[0] for desc in cursor.description]
df = pd.DataFrame(results, columns=coln)
print(df)

          batsman total_runs_in_death_overs
0        MS Dhoni                      7504
1      KA Pollard                      4574
2       HH Pandya                      4138
3       DA Miller                      4004
4      KD Karthik                      3826
5         V Kohli                      3670
6       RA Jadeja                      3566
7  AB de Villiers                      3354
8      AD Russell                      3208
9       RG Sharma                      2876
